## Análise de vendas da SuperStore usando SQL

### Definição do problema

Uma empresa de varejo de produtos  eletrônicos busca compreender melhor o desempenho das vendas. Para isso, coletou dados detalhados de vendas, incluindo informações sobre produtos, datas, preços, métodos de pagamento e localizações das lojas. 

O objetivo é extrair insights estratégicos dos dados, identificando padrões de compra, sazonalidade, oportunidades e desafios. Utilizando consultas SQL, a empresa pretende segmentar os clientes em categorias relevantes, a fim de compreender melhor seu comportamento e direcionar melhorias nas vendas.

Nesta análise, empregaremos o conjunto de dados obtido do Kaggle: 
https://www.kaggle.com/datasets/vivek468/superstore-dataset-final/data

In [1]:
# Precisamos instalar uma biblioteca para fazer a conexão com o MySQL através do Python: 
!pip install mysql-connector-python
# Precisamos importar as bibliotecas necessárias para o projeto:
import mysql.connector
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# Estabelecer uma conexão com o servidor MySQL criando um objeto de conexão e passando os parâmetros necessários:
conexao = mysql.connector.connect(
            host = "localhost",
            user = "seu_nome_usuario",
            password = "sua_senha",
            database = "nome_do_seu_schema"
)
# Testar a conexão com o MySQL:
if conexao.is_connected():
    print('Conexão ao MySQL bem-sucedida!')
else:
    print('Não foi possível realizar a conexão!')

# Criar um objeto cursor que irá executar as consultas:
cursor = conexao.cursor()

# Criando uma função que irá executar a query (consulta) no MySQL e retornar em formato de pandas DataFrame:
def show_query(query):
    cursor.execute(query)
    results = cursor.fetchall()
    df = pd.DataFrame(results, columns = [column[0] for column in cursor.description])
    return df

Conexão ao MySQL bem-sucedida!


### Análise exploratória

In [2]:
# Entenda as colunas e explore o conjunto de dados
query = '''
    SELECT * FROM superstore LIMIT 6
    
'''
show_query(query)

,ID_linha,ID_pedido,Data_do_pedido,Data_de_envio,Modo_de_envio,ID_cliente,Nome_do_cliente,Segmento,País,Cidade,...,Código_postal,Região,ID_produto,Categoria,Subcategoria,Nome_do_produto,Vendas,Quantidade,Desconto,Lucro
0,1,CA-2016-152156,2016-08-11,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,419.136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.00,219.582
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.00,68.714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,9.575.775,5,0.45,-383.031
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368,2,0.20,25.164
5,6,CA-2014-115812,2014-06-09,2014-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,...,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.86,7,0.00,141.694


In [3]:
# Receita total de vendas dos pedidos
query = '''
    SELECT ROUND(SUM(Vendas)) AS Total_de_Vendas FROM superstore
 
'''
show_query(query)

,Total_de_Vendas
0,1319969.0


- O total de Vendas dos pedidos é US$ 1.319.969

In [4]:
# Quantidade de pedidos presentes no conjunto de dados
query = '''
    SELECT COUNT(ID_pedido) AS Total_Pedidos FROM superstore

'''
show_query(query)

,Total_Pedidos
0,9279


- Este conjunto de dados contém um total de 9.279 pedidos.

In [5]:
# Quantidade total de pedidos vendidos
query = '''
    SELECT SUM(Quantidade) AS Quantidade_Total FROM superstore
    
'''
show_query(query)    

,Quantidade_Total
0,34095


- O número total de pedidos é 34.095

In [6]:
# Lucro total de todos os pedidos
query = '''
    SELECT ROUND(SUM(Lucro)) AS Total_de_lucro FROM superstore

'''
show_query(query) 

,Total_de_lucro
0,766505.0


- O lucro total é de US$ 766.505

In [7]:
# O desempenho das vendas e dos lucros ao longo dos anos
query = '''
    SELECT 
YEAR(Data_do_pedido) AS Ano,
SUM(Quantidade) AS Total_Quantidade, 
ROUND(SUM(Lucro), 2) AS Total_Lucro 
FROM 
superstore
GROUP BY YEAR(Data_do_pedido) 
ORDER BY Ano DESC

'''
show_query(query)

,Ano,Total_Quantidade,Total_Lucro
0,2017,11259,226781.76
1,2016,8886,201474.83
2,2015,7174,178091.83
3,2014,6776,160156.27


- Observa-se um aumento significativo tanto no número de pedidos quanto no lucro em 2017.

In [8]:
# Desconto e Margem de Lucro
query = '''
    SELECT ROUND(AVG(Desconto), 2) AS Desconto_Médio, ROUND(AVG(Lucro), 2) AS Lucro_Médio
    FROM superstore

'''
show_query(query)

,Desconto_Médio,Lucro_Médio
0,0.16,82.61


- O desconto médio foi de 16%, resultando em um lucro médio de US$ 82.61

In [9]:
# As regiões com as vendas mais altas
query = '''
    SELECT Região,
    YEAR(Data_do_pedido) AS Ano,
    ROUND(SUM(Vendas), 2) AS Total_Vendas,
    ROUND(SUM(Lucro), 2) AS Total_Lucro
    FROM superstore
    GROUP BY Região, YEAR(Data_do_pedido)
    ORDER BY Total_Vendas DESC

'''
show_query(query)

,Região,Ano,Total_Vendas,Total_Lucro
0,West,2017,145318.14,111077.68
1,East,2017,115470.96,56686.98
2,West,2016,112375.44,66890.88
3,West,2014,107767.14,73719.32
4,West,2015,97053.57,66409.81
5,East,2016,91519.75,54498.85
6,Central,2017,90018.28,18834.46
7,East,2015,82591.81,52091.74
8,Central,2016,81138.22,37449.35
9,Central,2015,68566.19,24311.84


- As vendas e lucros na região West se superaram consistentemente os números da região East ao longo dos anos, com destaque em 2017, onde as vendas totais na região West atingiram USD 145.318,14 , resultando em um lucro total de USD 111.077,68. Em comparação, na região East, as vendas totalizaram USD 115.470,96 , com um lucro total de USD 56.686,98 no mesmo ano. Os anos de 2016 e 2014 também mostraram um desempenho superior na região West em relação à região East.

East - Leste; West - Oeste

In [10]:
# A receita total de vendas de cada região
query = '''
    SELECT Região,
    ROUND(SUM(Vendas)) AS Receita_Total
    FROM superstore
    GROUP BY Região

'''
show_query(query)

,Região,Receita_Total
0,South,210475.0
1,West,462514.0
2,Central,294009.0
3,East,352971.0


- A análise revela que a receita total varia significativamente entre as regiões, sendo que a região West lidera com um montante de USD 462.514,00, seguida pela região East com USD 352.971,00, a região Central com USD 294.009,00, e por fim, a região South com um total de USD 210.475,00

East - Leste; West - Oeste; South - Sul

In [11]:
# A categoria de produtos mais solicitada
query = '''
    SELECT Categoria, COUNT(*) AS Número_de_encomendas 
    FROM superstore
    GROUP BY Categoria
    ORDER BY Número_de_encomendas DESC;

'''
show_query(query)

,Categoria,Número_de_encomendas
0,Office Supplies,5779
1,Furniture,1904
2,Technology,1596


- A categoria de produtos mais solicitada é Office Supplies (Material de escritório).

In [12]:
# O produto mais lucrativo
query = '''
    SELECT Nome_do_produto, ROUND(SUM(Lucro)) AS Total_Lucro
    FROM superstore 
    GROUP BY Nome_do_produto
    ORDER BY Total_Lucro DESC
    LIMIT 1

'''
show_query(query)

,Nome_do_produto,Total_Lucro
0,Easy-staple paper,9345.0


- O produto mais lucrativo é o Easy-staple paper (Papel de grampeamento fácil), com um lucro de US$ 9.345,00

In [13]:
# As cidades com o maior lucro
query = '''
    SELECT País, Cidade, 
    SUM(Quantidade) AS Total_Vendido,
    ROUND(SUM(Vendas), 2) AS Total_Vendas,
    ROUND(SUM(Lucro), 2) AS Total_Lucro
    FROM superstore 
    GROUP BY País, Cidade
    ORDER BY Total_Lucro DESC
    LIMIT 5

'''
show_query(query)

,País,Cidade,Total_Vendido,Total_Vendas,Total_Lucro
0,United States,New York City,2934,122037.39,109226.06
1,United States,Los Angeles,2630,114241.48,87405.67
2,United States,San Francisco,1790,71918.48,59035.53
3,United States,Seattle,1415,61342.83,50439.69
4,United States,Detroit,376,28813.67,24938.23


- Essas cidades se destacam por seu excepcional desempenho em termos de lucratividade nas vendas de produtos nos Estados Unidos. Um exemplo é New York City, que registrou a venda de 2.934 unidades, gerando um lucro total de US$ 109.226,06

In [14]:
# Os 10 primeiros clientes classificados de acordo com seus gastos totais
query = '''
    SELECT Nome_do_cliente,
    SUM(Vendas) AS Gastos_Totais
    FROM superstore
    GROUP BY Nome_do_cliente
    ORDER BY Gastos_Totais DESC
    LIMIT 10

'''
show_query(query)

,Nome_do_cliente,Gastos_Totais
0,Adrian Barton,12154.556
1,Ken Lonsdale,11081.101
2,Laura Armstrong,8673.222
3,Clay Ludtke,8069.616
4,Anna Häberlin,7888.294
5,Helen Wasserman,7374.374
6,Zuschuss Carroll,6936.915
7,Seth Vernon,6851.187
8,Greg Tran,6771.814
9,Natalie Fritzler,6522.832


- Os 10 primeiros clientes, classificados de acordo com seus gastos totais, são liderados por Adrian Barton, cujos gastos totais atingem USD 12.154,56. Em seguida, temos Ken Lonsdale, com gastos totais de USD 11.081,10, seguido por Laura Armstrong, com USD 8.673,22, e Clay Ludtke, com USD 8.069,62, entre outros. Esses clientes demonstram um alto envolvimento e contribuição para o volume de vendas da empresa.

In [15]:
# A quantidade de pedidos realizados em cada ano
query = '''
    SELECT YEAR(Data_do_pedido) AS Ano,
    COUNT(*) AS Total_Pedidos
    FROM superstore
    GROUP BY YEAR(Data_do_pedido)
    ORDER BY Ano

'''
show_query(query)

,Ano,Total_Pedidos
0,2014,1849
1,2015,1952
2,2016,2398
3,2017,3080


- A quantidade de pedidos realizados aumentou gradualmente ao longo dos anos. Em 2014, foram registrados 1.849 pedidos, seguidos por 1.952 em 2015, 2.398 em 2016 e 3.080 em 2017. Esse aumento progressivo sugere um crescimento consistente nas operações da empresa ao longo do período analisado.

In [16]:
# O tempo médio decorrido do pedido até a data de envio para cada opção de envio
query = '''
    SELECT Modo_de_envio,
    AVG(DATEDIFF(Data_de_envio, Data_do_pedido)) AS Tempo_Medio_Entrega
    FROM superstore
    GROUP BY Modo_de_envio

'''
show_query(query)

,Modo_de_envio,Tempo_Medio_Entrega
0,Second Class,3.2910
1,Standard Class,5.0092
2,First Class,2.1783
3,Same Day,0.0478


- Para cada opção de envio, o tempo médio decorrido do pedido até a data de envio varia. O envio Same Day apresenta o menor tempo médio de 0.0478 dias, seguido pelo First Class com 2.1783 dias. Enquanto isso, o Second Class tem um tempo médio de 3.2910 dias, e o Standard Class possui o maior tempo médio de entrega, com 5.0092 dias. Esses dados são importantes para entender o desempenho e a eficiência de cada opção de envio oferecida pela empresa.

Second Class - Segunda classe; Standard Class - Classe padrão; First Class	- Primeira classe; Same Day - Mesmo dia 

In [17]:
# O desconto médio aplicado nas encomendas
query = '''
    SELECT ROUND(AVG(Desconto), 2) AS Desconto_Medio
    FROM superstore

'''
show_query(query)

,Desconto_Medio
0,0.16


- O desconto é de 16% nas encomendas.

In [18]:
# Padrões de vendas ao longo do ano: Segmentos e Categorias
query = '''
    SELECT 
        YEAR(Data_do_pedido) AS Ano,
        Segmento,
        Categoria,
        ROUND(AVG(Vendas), 2) AS Vendas_Medio
    FROM (
        SELECT
            Data_do_pedido,
            Segmento,
            Categoria,
            Vendas,
            AVG(Vendas) OVER (PARTITION BY Segmento, Categoria ORDER BY Data_do_pedido 
            ROWS BETWEEN 2 PRECEDING AND 2 FOLLOWING) AS Média_Móvel_Vendas
        FROM superstore
    ) AS subquery
    GROUP BY Ano, Segmento, Categoria
    ORDER BY Ano, Segmento, Categoria

'''
show_query(query)

,Ano,Segmento,Categoria,Vendas_Medio
0,2014,Consumer,Furniture,270.76
1,2014,Consumer,Office Supplies,86.49
2,2014,Consumer,Technology,237.17
3,2014,Corporate,Furniture,257.02
4,2014,Corporate,Office Supplies,82.47
5,2014,Corporate,Technology,207.62
6,2014,Home Office,Furniture,247.22
7,2014,Home Office,Office Supplies,70.10
8,2014,Home Office,Technology,347.79
9,2015,Consumer,Furniture,292.87


- Os padrões de vendas ao longo dos anos revelam que, em média, o Segmento Consumer demonstra maior volume de vendas, com destaque para a Categoria Technology, enquanto o Segmento Corporate apresenta resultados consistentes e o Segmento Home Office tende a variar mais, especialmente na categoria Furniture.

In [19]:
# Média de Desconto por Categoria de Produto
query = '''
    SELECT 
        Categoria,
        ROUND(AVG(Desconto), 2) AS Desconto_Medio
    FROM
        superstore
    GROUP BY
        Categoria  

'''
show_query(query)

,Categoria,Desconto_Medio
0,Furniture,0.17
1,Office Supplies,0.16
2,Technology,0.13


- As categorias de produtos apresentam médias de desconto distintas: Furniture com 17%, Office Supplies com 16% e Technology com 13%. Esses dados são cruciais para entender as estratégias de precificação e desconto em cada categoria e podem orientar decisões futuras de marketing e vendas.

In [20]:
# Detalhes de Preços, Descontos e perdas por Produto
query = '''
    SELECT 
        ID_produto,
        Nome_do_produto,
        Vendas AS Preco_Original,
        Desconto,
        ROUND((Desconto / Vendas) * 100, 2) AS Percentual_Desconto,
        ROUND(Vendas - (Desconto / 1 - (Desconto / Vendas)), 2) AS Perda
    FROM
        superstore
    ORDER BY Perda DESC
    LIMIT 10

'''
show_query(query)

,ID_produto,Nome_do_produto,Preco_Original,Desconto,Percentual_Desconto,Perda
0,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,9892.74,0.0,0.00,9892.74
1,OFF-SU-10000151,High Speed Automatic Electric Letter Opener,8187.65,0.0,0.00,8187.65
2,TEC-MA-10000984,Okidata MB760 Printer,4476.8,0.2,0.00,4476.60
3,OFF-SU-10002881,Martin Yale Chadless Opener Electric Letter Op...,4164.05,0.0,0.00,4164.05
4,OFF-SU-10002881,Martin Yale Chadless Opener Electric Letter Op...,4164.05,0.0,0.00,4164.05
5,FUR-BO-10002213,DMI Eclipse Executive Suite Bookcases,4007.84,0.2,0.00,4007.64
6,TEC-MA-10002927,Canon imageCLASS MF7460 Monochrome Digital Las...,3991.98,0.0,0.00,3991.98
7,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,3504.9,0.0,0.00,3504.90
8,TEC-MA-10000984,Okidata MB760 Printer,3357.6,0.2,0.01,3357.40
9,TEC-MA-10000864,Cisco 9971 IP Video Phone Charcoal,3080,0.0,0.00,3080.00


- O produto GBC Ibimaster 500 Manual ProClick Binding System possui um preço original de USD 9.892,74, sem desconto aplicado e sem perda associada. Já o produto Okidata MB760 Printer tem um preço original de USD 4.476,80, com um desconto de 20% aplicado, resultando em uma perda de USD 4.476,60. Esses dados são cruciais para entender o impacto dos descontos nas margens de lucro e podem ajudar na otimização das estratégias de preços.

In [21]:
# As tendências sazonais e mensais para identificar padrões de vendas ao longo do ano
query = '''
    SELECT 
        YEAR(Data_do_pedido) AS Ano,
        MONTH(Data_do_pedido) AS Mes,
        ROUND(SUM(Vendas), 2) AS Total_Vendas
    FROM
        superstore
    GROUP BY Ano, Mes
    ORDER BY Ano, Mes

'''
show_query(query)

,Ano,Mes,Total_Vendas
0,2014,1,8747.26
1,2014,2,4519.89
2,2014,3,17406.15
3,2014,4,22381.38
4,2014,5,14975.45
5,2014,6,17542.53
6,2014,7,27662.72
7,2014,8,16343.96
8,2014,9,31659.02
9,2014,10,22144.48


- Observa-se um aumento nas vendas nos meses de Novembro e Dezembro em todos os anos, indicando um pico de atividade durante a temporada de festas. Além disso, pode-se notar variações mensais ao longo do ano, com flutuações nas vendas em diferentes meses. Essas informações são valiosas para planejar estratégias de marketing sazonais e ajustar o estoque de acordo com a demanda sazonal.

In [22]:
# Avaliação do impacto dos descontos nas vendas e lucratividade
query = '''
    SELECT
        IF(Desconto > 0, 'Com Desconto', 'Sem Desconto') AS Tipo_Desconto,
        ROUND(SUM(Vendas), 2) AS Total_Vendas,
        ROUND(SUM(Lucro), 2) AS Total_Lucro
    FROM
        superstore
    GROUP BY Tipo_Desconto

'''
show_query(query)

,Tipo_Desconto,Total_Vendas,Total_Lucro
0,Sem Desconto,604836.86,680353.7
1,Com Desconto,715132.31,86151.0


- A avaliação mostra que a estratégia de aplicação de descontos teve um impacto significativo nas vendas e lucratividade. Quando comparadas as vendas sem desconto, que totalizaram USD 604.836,86, com as vendas com desconto, que atingiram USD 715.132,31, observa-se um aumento notável nas vendas. No entanto, apesar do aumento nas vendas com desconto, a lucratividade foi menor, com um total de lucro de USD 680.353,70 sem desconto e apenas USD 86.151,00 com desconto. Isso sugere que, embora os descontos possam impulsionar as vendas, eles podem reduzir a lucratividade geral da empresa.

In [23]:
# Segmentação dos clientes com base em seu comportamento de compra
query = '''
    SELECT 
        CASE 
            WHEN Gastos_Totais < 1000 THEN 'Cliente Regular'
            WHEN Gastos_Totais >= 1000 AND Gastos_Totais < 5000 THEN 'Cliente Premium'
            ELSE 'Cliente VIP'
        END AS Segmento_Cliente,
        COUNT(*) AS Numero_de_Clientes,
        ROUND(SUM(Gastos_Totais), 2) AS Receita_Total
    FROM (
        SELECT 
            Nome_do_cliente,
            SUM(Vendas) AS Gastos_Totais
        FROM
            superstore
        GROUP BY Nome_do_cliente
    ) AS subquery
    GROUP BY Segmento_Cliente    

'''
show_query(query)

,Segmento_Cliente,Numero_de_Clientes,Receita_Total
0,Cliente Premium,473,1005852.78
1,Cliente Regular,298,169365.25
2,Cliente VIP,21,144751.14


- A segmentação dos clientes com base em seu comportamento de compra revela que a maioria dos clientes pertence ao segmento Premium, em seguida o segmento Regular e por fim, o segmento VIP. Essas informações são essenciais para a importância de cada segmento na composição da receita e na compreensão do perfil dos clientes, direcionando as estratégias de atendimento de forma mais eficaz.  

In [24]:
# A rentabilidade dos produtos considerando os custos associados
query = '''
    SELECT 
        Nome_do_produto,
        ROUND(SUM(Vendas), 2) AS Receita_Total,
        ROUND(SUM(Lucro), 2) AS Lucro_Total,
        ROUND(SUM(Lucro) / SUM(Vendas), 2) AS Margem_Lucro
    FROM
        superstore
    GROUP BY Nome_do_produto
    ORDER BY Margem_Lucro DESC    
    LIMIT 15

'''
show_query(query)

,Nome_do_produto,Receita_Total,Lucro_Total,Margem_Lucro
0,Apple iPhone 5,1.04,909.76,875.61
1,Smead Adjustable Mobile File Trolley with Lock...,1.01,880.30,875.05
2,Tennsco 16-Compartment Lockers with Coat Rack,3.63,1814.09,500.30
3,Canon Color ImageCLASS MF8580Cdw Wireless Lase...,1.01,431.99,428.99
4,Logitech diNovo Edge Keyboard,1.40,524.98,375.25
5,Personal Creations Ink Jet Cards and Labels,321.44,1575.06,4.90
6,Avery 478,88.38,433.06,4.90
7,Color-Coded Legal Exhibit Labels,14.73,72.18,4.90
8,Rediform S.O.S. Phone Message Books,19.92,97.61,4.90
9,Green Bar Computer Printout Paper,549.60,2693.04,4.90


- Existe uma variação significativa nas margens de lucro. Por exemplo, produtos como "Apple iPhone 5" e "Smead Adjustable Mobile File Trolley with Lock" apresentam margens de lucro bastante elevadas, atingindo USD 875,61 e USD 875,05, respectivamente. Em contraste, itens como "Personal Creations Ink Jet Cards and Labels" e "Avery 478" possuem margens de lucro mais baixas, totalizando apenas USD 4,90 cada. Esses dados são cruciais para identificar quais produtos são mais lucrativos e podem orientar decisões relacionadas à precificação, promoções e gestão de estoque.

In [25]:
# Avaliação da taxa de retenção de clientes ao longo do tempo
query = '''
    SELECT 
        YEAR(Data_do_pedido) AS Ano,
        COUNT(DISTINCT Nome_do_cliente) AS Clientes_unicos
    FROM
        superstore
    GROUP BY Ano    

'''
show_query(query)

,Ano,Clientes_unicos
0,2014,579
1,2015,561
2,2016,632
3,2017,688


- Revela um aumento constante no número de clientes únicos de 2014 a 2017. O número de clientes únicos passou de 579 em 2014 para 688 em 2017, indicando um crescimento consistente da base de clientes ao longo desses anos. Esses dados sugerem uma tendência positiva de retenção de clientes, o que pode refletir a eficácia das estratégias de fidelização e satisfação do cliente implementadas pela empresa.

### Conclusões

#### Insights 
- Performance financeira global: As vendas totais, o número de pedidos e o lucro total aumentaram ao longo dos anos, com um destaque especial para o aumento significativo em 2017.

- Desempenho regional: A região West superou consistentemente a região East em vendas e lucros, especialmente notável em 2017.

- Categorias e produtos: A categoria de produtos mais solicitada é Office Supplies, e o produto mais lucrativo é o Easy-staple paper. Além disso, produtos específicos têm margens de lucro variadas, o que pode influenciar estratégias de precificação.

- Clientes: Os 10 principais clientes contribuem significativamente para o volume de vendas da empresa, evidenciando a importância do relacionamento com os clientes mais lucrativos.

- Evolução ao longo do tempo: O número de pedidos e clientes únicos aumentou consistentemente ao longo dos anos, sugerindo um crescimento saudável da empresa.

- Impacto dos descontos: Embora os descontos possam impulsionar as vendas, eles podem reduzir a lucratividade global da empresa, como evidenciado pela comparação entre vendas com e sem desconto.

- Comportamento sazonal: As vendas mostram um pico nos meses de Novembro e Dezembro, indicando uma forte temporada de festas. Esse padrão sazonal pode ser usado para ajustar estratégias de marketing e estoque.

- Segmentação de clientes: A maioria dos clientes pertence ao segmento Premium, seguido pelo segmento Regular e, por último, o segmento VIP. Isso destaca a importância de cada segmento na receita da empresa.

#### Recomendações

- Otimização de Estratégias de Desconto: Considerando o impacto significativo dos descontos nas vendas e na lucratividade, a empresa pode querer revisar suas estratégias de desconto. Isso pode incluir a implementação de descontos mais direcionados e segmentados, bem como a análise do retorno sobre o investimento em descontos.

- Foco em Produtos Lucrativos: Identificar e promover produtos com margens de lucro mais elevadas, como o Easy-staple paper, pode ajudar a impulsionar a rentabilidade geral da empresa. Isso pode envolver ajustes na estratégia de precificação, promoções específicas para esses produtos ou até mesmo o desenvolvimento de novos produtos que se alinhem com os de alta rentabilidade.

- Expansão Geográfica e Foco Regional: Dada a disparidade de desempenho entre as regiões, a empresa pode querer investir mais em áreas onde há potencial para crescimento, como a região East. Isso pode envolver estratégias de marketing mais direcionadas, expansão da presença física ou online e melhorias nos canais de distribuição.

- Melhoria da Experiência do Cliente: Investir em programas de fidelização de clientes e melhorar a experiência geral do cliente pode ajudar a reter clientes existentes e atrair novos. Isso pode incluir melhorias no atendimento ao cliente, personalização de comunicações e ofertas, e simplificação do processo de compra.

- Gestão de Estoque e Planejamento Sazonal: Considerando os padrões sazonais nas vendas, a empresa deve garantir que seu estoque esteja alinhado com a demanda esperada. Isso pode envolver ajustes nas quantidades de estoque por produto e a introdução de novas linhas de produtos sazonais.

- Análise de Segmentação de Clientes: Aprofundar a compreensão dos diferentes segmentos de clientes pode ajudar a empresa a direcionar suas estratégias de marketing e atendimento de forma mais eficaz. Isso pode envolver o desenvolvimento de campanhas personalizadas, ofertas exclusivas e programas de recompensa específicos para cada segmento.

- Monitoramento Contínuo e Análise de Dados: Estabelecer um processo contínuo de monitoramento e análise de dados pode ajudar a empresa a identificar tendências emergentes, oportunidades de melhoria e áreas de risco. Isso pode envolver o uso de ferramentas analíticas avançadas e a criação de painéis de controle personalizados para acompanhar os principais indicadores de desempenho.